In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )
import json
import requests
import pandas as pd
from decimal import Decimal
from carbon.carbon_order_ui import CarbonOrderUI
from carbon.sdk.carbonsdk import EncodedOrder
from benchmark import alphaxutils
import pprint

Carbon Version v2.4.1-BETA1 (17/Mar/2023)


# Carbon Simulation - Demo TestSuite

This book is for creating test scenarios for Tenderly Contracts

#### Basic token getter for decimals

In [2]:
r = requests.get('https://tokens.coingecko.com/uniswap/all.json')
cgtokens = pd.DataFrame(eval(r.text)['tokens'])

tokenDecimals = {}
tokenDecimals['ETH'] = 18
for i in cgtokens.index:
    tokenDecimals[cgtokens.symbol[i]] = int(cgtokens.decimals[i])

## Enter tkn0 and tkn1

In [3]:
tkn0 = "ETH"  #base token
tkn1 = "USDC" #quote token

token0Decimals = tokenDecimals[tkn0]
token1Decimals = tokenDecimals[tkn1]
delta_tokenDecimals = token1Decimals - token0Decimals
print(tkn0, token0Decimals)
print(tkn1, token1Decimals)

ETH 18
USDC 6


## Initialize Simulator instance and Add strategies

In [4]:
def reset_Sim():
    Sim = CarbonSimulatorUI(pair=f"{tkn0}/{tkn1}", verbose=False, matching_method='fast')  # important to denote fast matching. pairs are specified base/quote and strategy is entered in quote per base units
    for i in range(2):
        Sim.add_strategy(
            tkn=f'{tkn0}',          # specify sell side tkn

            pbuy_end  =1000,        # price to stop buying in quote per base units
            pbuy_start=1700,        # price to start buying in quote per base units
            amt_buy   =10000,       # liqudity amount of paired tkn to buy
            
            psell_start=2000,       # price to start selling in quote per base units
            psell_end  =2500,       # price to stop selling in quote per base units
            amt_sell   =20,         # liqudity amount of tkn to sell
            
            pair=f"{tkn0}/{tkn1}",  # pair
        )
    return(Sim)

Sim = reset_Sim()

## Show all strategies

In [5]:
Sim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,20.0,20.0,ETH,False,2000.0,2500.0,2000.0,USDC per ETH,1,
1,1,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1700.0,1000.0,1700.0,USDC per ETH,0,
2,2,ETHUSDC,ETH,20.0,20.0,ETH,False,2000.0,2500.0,2000.0,USDC per ETH,3,
3,3,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1700.0,1000.0,1700.0,USDC per ETH,2,


#### Liquidity depth - acting on current Sim state
- dyfromp_f(price in quote per base units)

In [6]:
sum([Sim.state()['orderuis'][key].dyfromp_f(2500) for key in list(Sim.state()['orderuis'].keys())[::2]])

40.0

In [7]:
sum([Sim.state()['orderuis'][key].dyfromp_f(1500) for key in list(Sim.state()['orderuis'].keys())[1::2]])

5206.390496623133

## Fetch orders and convert to contract variables

In [8]:
def get_contract_orders(Sim):
    order_dict = {}
    for key in Sim.orders.keys():
        test_order = CarbonOrderUI.from_order(Sim.orders[key])
        test_order.pair.set_decimals(dct=tokenDecimals)
        order_dict[key] = EncodedOrder.encode_yzAB(test_order.yzABS(sx=48).astuple[:4], token=test_order.tkn)
    return(order_dict)

In [9]:
initial_state = get_contract_orders(Sim)
pprint.pprint(initial_state)

{0: EncodedOrder(token='ETH',
                 y=20000000000000000000,
                 z=20000000000000000000,
                 A=3539924399406985,
                 B=4393923342499840),
 1: EncodedOrder(token='USDC',
                 y=10000000000,
                 z=10000000000,
                 A=2704490291,
                 B=8901020307),
 2: EncodedOrder(token='ETH',
                 y=20000000000000000000,
                 z=20000000000000000000,
                 A=3539924399406985,
                 B=4393923342499840),
 3: EncodedOrder(token='USDC',
                 y=10000000000,
                 z=10000000000,
                 A=2704490291,
                 B=8901020307)}


## Do Trades in each direction

## Enter an amount

In [10]:
USDC_amount_1 = 50000

## APP - "Buy ETH with USDC" box

In [11]:
# Enter 1 USDC into the "BUY ETH for USDC" box
tbSou10 = Sim.amm_buys(f"{tkn1}", USDC_amount_1, execute=True, support_partial=False) # route_trade_by_source  #note the variable name "tbSou10" == {tradeBySource}{sourceToken}{targetToken}. e.g. in the TokensTraded event tkn1 should be the source token
tbSou10['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,False,None,20.000000,ETH,44721.35955,USDC,ETHUSDC,0,1,2212.090414,USDC per ETH
0,0.1,0,1,route #2,False,True,False,None,2.603054,ETH,5278.64045,USDC,ETHUSDC,2,1,2212.090414,USDC per ETH
0,0,0,A,AMM sells 23ETH buys 50000USDC,True,True,False,None,22.603054,ETH,50000.00000,USDC,ETHUSDC,"[0, 2]",2,2212.090414,USDC per ETH


In [12]:
tbSou10['trades'][['subid', 'amt1', 'tkn1', 'amt2', 'tkn2']]

,subid,amt1,tkn1,amt2,tkn2
0,0,20.000000,ETH,44721.35955,USDC
0,1,2.603054,ETH,5278.64045,USDC
0,A,22.603054,ETH,50000.00000,USDC


In [13]:
tbSou10_orders = Sim.state()['orders']
tbSou10_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,20.00000,0.000000,ETH,False,2000.0,2500.0,2500.000000,USDC per ETH,1,
1,1,ETHUSDC,USDC,54721.35955,54721.359550,USDC,False,1700.0,1000.0,1700.000000,USDC per ETH,0,
2,2,ETHUSDC,ETH,20.00000,17.396946,ETH,False,2000.0,2500.0,2056.116293,USDC per ETH,3,
3,3,ETHUSDC,USDC,15278.64045,15278.640450,USDC,False,1700.0,1000.0,1700.000000,USDC per ETH,2,


In [14]:
state1 = get_contract_orders(Sim)
pprint.pprint(state1)

{0: EncodedOrder(token='ETH',
                 y=0,
                 z=20000000000000000000,
                 A=3539924399406985,
                 B=4393923342499840),
 1: EncodedOrder(token='USDC',
                 y=54721359549,
                 z=54721359549,
                 A=2704490291,
                 B=8901020307),
 2: EncodedOrder(token='ETH',
                 y=17396945587640000512,
                 z=20000000000000000000,
                 A=3539924399406985,
                 B=4393923342499840),
 3: EncodedOrder(token='USDC',
                 y=15278640450,
                 z=15278640450,
                 A=2704490291,
                 B=8901020307)}


### Reset Sim before each test

In [15]:
Sim = reset_Sim()

In [16]:
ETH_amount_1 = 35.2

In [17]:
# Enter 1 ETH into the "BUY ETH for USDC" box
tbTar10 = Sim.amm_sells(f"{tkn0}", ETH_amount_1, execute=True, support_partial=False) # route_trade_by_target
tbTar10['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,False,None,20.0,ETH,44721.359550,USDC,ETHUSDC,0,1,2209.468536,USDC per ETH
0,0.1,0,1,route #2,False,True,False,None,15.2,ETH,33051.932904,USDC,ETHUSDC,2,1,2209.468536,USDC per ETH
0,0,0,A,AMM sells 35ETH buys 77773USDC,True,True,True,None,35.2,ETH,77773.292454,USDC,ETHUSDC,"[0, 2]",2,2209.468536,USDC per ETH


In [18]:
tbTar10['trades'][['subid', 'amt1', 'tkn1', 'amt2', 'tkn2']]

,subid,amt1,tkn1,amt2,tkn2
0,0,20.0,ETH,44721.359550,USDC
0,1,15.2,ETH,33051.932904,USDC
0,A,35.2,ETH,77773.292454,USDC


In [19]:
Sim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,20.000000,0.000000,ETH,False,2000.0,2500.0,2500.000000,USDC per ETH,1,
1,1,ETHUSDC,USDC,54721.359550,54721.359550,USDC,False,1700.0,1000.0,1700.000000,USDC per ETH,0,
2,2,ETHUSDC,ETH,20.000000,4.800000,ETH,False,2000.0,2500.0,2364.158303,USDC per ETH,3,
3,3,ETHUSDC,USDC,43051.932904,43051.932904,USDC,False,1700.0,1000.0,1700.000000,USDC per ETH,2,


In [20]:
state2 = get_contract_orders(Sim)
pprint.pprint(state2)

{0: EncodedOrder(token='ETH',
                 y=0,
                 z=20000000000000000000,
                 A=3539924399406985,
                 B=4393923342499840),
 1: EncodedOrder(token='USDC',
                 y=54721359549,
                 z=54721359549,
                 A=2704490291,
                 B=8901020307),
 2: EncodedOrder(token='ETH',
                 y=4800000000000001024,
                 z=20000000000000000000,
                 A=3539924399406985,
                 B=4393923342499840),
 3: EncodedOrder(token='USDC',
                 y=43051932903,
                 z=43051932903,
                 A=2704490291,
                 B=8901020307)}


## APP - "Sell ETH with USDC" box

### Reset Sim before each test

In [21]:
Sim = reset_Sim()

In [22]:
ETH_amount_2 = 9.2

In [23]:
# Enter 1 ETH into the "SELL ETH for USDC" box
tbSou01 = Sim.amm_buys(f"{tkn0}", ETH_amount_2, execute=True, support_partial=False) # route_trade_by_source
tbSou01['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,False,None,10000.000000,USDC,7.66965,ETH,ETHUSDC,1,1,1353.574549,USDC per ETH
0,0.1,0,1,route #3,False,True,False,None,2452.885854,USDC,1.53035,ETH,ETHUSDC,3,1,1353.574549,USDC per ETH
0,0,0,A,AMM sells 12453USDC buys 9ETH,True,True,True,None,12452.885854,USDC,9.20000,ETH,ETHUSDC,"[1, 3]",2,1353.574549,USDC per ETH


In [24]:
tbSou01['trades'][['subid', 'amt1', 'tkn1', 'amt2', 'tkn2']]

,subid,amt1,tkn1,amt2,tkn2
0,0,10000.000000,USDC,7.66965,ETH
0,1,2452.885854,USDC,1.53035,ETH
0,A,12452.885854,USDC,9.20000,ETH


In [25]:
Sim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,27.66965,27.669650,ETH,False,2000.0,2500.0,2000.000000,USDC per ETH,1,
1,1,ETHUSDC,USDC,10000.00000,0.000000,USDC,False,1700.0,1000.0,1000.000000,USDC per ETH,0,
2,2,ETHUSDC,ETH,21.53035,21.530350,ETH,False,2000.0,2500.0,2000.000000,USDC per ETH,3,
3,3,ETHUSDC,USDC,10000.00000,7547.114146,USDC,False,1700.0,1000.0,1511.207696,USDC per ETH,2,


In [26]:
state3 = get_contract_orders(Sim)
pprint.pprint(state3)

{0: EncodedOrder(token='ETH',
                 y=27669649888473997312,
                 z=27669649888473997312,
                 A=3539924399406985,
                 B=4393923342499840),
 1: EncodedOrder(token='USDC', y=0, z=10000000000, A=2704490291, B=8901020307),
 2: EncodedOrder(token='ETH',
                 y=21530350111526002688,
                 z=21530350111526002688,
                 A=3539924399406985,
                 B=4393923342499840),
 3: EncodedOrder(token='USDC',
                 y=7547114146,
                 z=10000000000,
                 A=2704490291,
                 B=8901020307)}


### Reset Sim before each test

In [27]:
Sim = reset_Sim()

In [28]:
USDC_amount_2 = 15770

In [29]:
# Enter 1 USDC into the "SELL ETH for USDC" box
tbTar01 = Sim.amm_sells(f"{tkn1}", USDC_amount_2, execute=True, support_partial=False) # route_trade_by_target
tbTar01['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,False,None,10000.0,USDC,7.669650,ETH,ETHUSDC,1,1,1360.533333,USDC per ETH
0,0.1,0,1,route #3,False,True,False,None,5770.0,USDC,3.921393,ETH,ETHUSDC,3,1,1360.533333,USDC per ETH
0,0,0,A,AMM sells 15770USDC buys 12ETH,True,True,True,None,15770.0,USDC,11.591043,ETH,ETHUSDC,"[1, 3]",2,1360.533333,USDC per ETH


In [30]:
tbTar01['trades'][['subid', 'amt1', 'tkn1', 'amt2', 'tkn2']]

,subid,amt1,tkn1,amt2,tkn2
0,0,10000.0,USDC,7.669650,ETH
0,1,5770.0,USDC,3.921393,ETH
0,A,15770.0,USDC,11.591043,ETH


In [31]:
Sim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,27.669650,27.669650,ETH,False,2000.0,2500.0,2000.0000,USDC per ETH,1,
1,1,ETHUSDC,USDC,10000.000000,0.000000,USDC,False,1700.0,1000.0,1000.0000,USDC per ETH,0,
2,2,ETHUSDC,ETH,23.921393,23.921393,ETH,False,2000.0,2500.0,2000.0000,USDC per ETH,3,
3,3,ETHUSDC,USDC,10000.000000,4230.000000,USDC,False,1700.0,1000.0,1273.5676,USDC per ETH,2,


In [32]:
state4 = get_contract_orders(Sim)
pprint.pprint(state4)

{0: EncodedOrder(token='ETH',
                 y=27669649888473997312,
                 z=27669649888473997312,
                 A=3539924399406985,
                 B=4393923342499840),
 1: EncodedOrder(token='USDC', y=0, z=10000000000, A=2704490291, B=8901020307),
 2: EncodedOrder(token='ETH',
                 y=23921392839566000128,
                 z=23921392839566000128,
                 A=3539924399406985,
                 B=4393923342499840),
 3: EncodedOrder(token='USDC',
                 y=4230000000,
                 z=10000000000,
                 A=2704490291,
                 B=8901020307)}
